# Cellular Automata Rules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import os
from datetime import datetime

colors = {
    'Blues': mcolors.LinearSegmentedColormap.from_list("Blues", ['#052E5A', '#93CCEA']),
    'CoolCyans': mcolors.LinearSegmentedColormap.from_list("CoolCyans", ['#009DAF', '#43C7D6']),
    'CoolGrays': mcolors.LinearSegmentedColormap.from_list("CoolGrays", ['#6C757D', '#B8C2CC']),    
    'CoolOranges': mcolors.LinearSegmentedColormap.from_list("CoolOranges", ['#E08300', '#F7C84C']),
    'CoolYellows': mcolors.LinearSegmentedColormap.from_list("CoolYellows", ['#C4A000', '#F3D547']),
    'CyanBlues': mcolors.LinearSegmentedColormap.from_list("CyanBlues", ['#006380', '#00A0B0']),
    'CyanGreens': mcolors.LinearSegmentedColormap.from_list("CyanGreens", ['#007046', '#26A66E']),
    'CyanGrays': mcolors.LinearSegmentedColormap.from_list("CyanGrays", ['#5B6166', '#8D959A']),
    'CyanYellows': mcolors.LinearSegmentedColormap.from_list("CyanYellows", ['#AA6600', '#FFBE00']),
    'DeepCyans': mcolors.LinearSegmentedColormap.from_list("DeepCyans", ['#008B8B', '#00CED1']),
    'DeepGrays': mcolors.LinearSegmentedColormap.from_list("DeepGrays", ['#A9A9A9', '#808080']),
    'DeepYellows': mcolors.LinearSegmentedColormap.from_list("DeepYellows", ['#DAA520', '#FFD700']),
    'GrayGrays': mcolors.LinearSegmentedColormap.from_list("GrayGrays", ['#363636', '#999999']),
    'LightBlues': mcolors.LinearSegmentedColormap.from_list("LightBlues", ['#ADD8E6', '#E0FFFF']),
    'LightGrays': mcolors.LinearSegmentedColormap.from_list("LightGrays", ['#D3D3D3', '#F5F5F5']),
    'WarmCyans': mcolors.LinearSegmentedColormap.from_list("WarmCyans", ['#1E7F9C', '#6BB7C8']),
    'WarmGreens': mcolors.LinearSegmentedColormap.from_list("WarmGreens", ['#2E723D', '#82BA5F']),
    'WarmGrays': mcolors.LinearSegmentedColormap.from_list("WarmGrays", ['#5B6166', '#AEB6BD']),
    'WarmYellows': mcolors.LinearSegmentedColormap.from_list("WarmYellows", ['#C48400', '#FFB500']),
    'Yellows': mcolors.LinearSegmentedColormap.from_list("Yellows", ['#CCCC00', '#FFFF00'])
}

def cellular_automaton_function(rule_num, num_cells=257):
    # Converts the rule number to binary and saves it in an array
    rule_bin = [int(x) for x in np.binary_repr(rule_num, width=8)]
    rule = np.array(rule_bin[::-1])  # it is inverted to match our definitions

    num_generations = num_cells // 2

    # We define the initial condition (all cells are off except the middle one)
    cells = np.zeros(num_cells)
    cells[num_cells // 2] = 1

    # We create a matrix to save all the generations
    generations = np.zeros((num_generations, num_cells))

    # We run the cellular automaton
    for i in range(num_generations):
        generations[i, :] = cells

        # Create an array for the next generation
        new_cells = np.zeros(num_cells)

        # We calculate the new cells according to the rule
        for j in range(1, num_cells - 1):
            # We get the current state of the cell and its neighbors
            state = cells[j - 1:j + 2]

            # We convert the state to a binary number and apply the rule
            idx = int("".join(str(int(x)) for x in state), 2)
            new_cells[j] = rule[idx]

        # The new generation becomes the current one
        cells = new_cells

    # We select a color combination randomly
    color_name = random.choice(list(colors.keys()))    
    colormap = colors[color_name]

    # We draw the cellular automaton with the random color combination
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.imshow(generations, cmap=colormap)

    # We add the white box without borders
    message = f"Rule {rule_num} \n Art of Cellular Automata"

    bbox_props = dict(boxstyle="round,pad=0.3", fc="white", lw=0)  # We remove the parameter ec="black" to remove the borders
    ax.text(num_cells//2, num_cells//2 - 12, message, fontsize=40, ha='center', va='center', bbox=bbox_props, fontname='Sans-serif')

    plt.axis('off')
    plt.tight_layout()  # Adjust the margins of the figure

    # Create directory "imgs" if it doesn't exist
    if not os.path.exists("imgs"):
        os.makedirs("imgs")

    # Get current time
    now = datetime.now()
    hhmmss = now.strftime("%H%M%S")

    # Filename
    filename = f"imgs/Rule{rule_num}{hhmmss}"

    # Save image in .png format
    plt.savefig(f"{filename}.png", format="png", dpi=300, bbox_inches='tight')

    # Save image in .pdf format
    plt.savefig(f"{filename}.pdf", format="pdf", bbox_inches='tight')

    plt.show()


In [ ]:
cellular_automaton_function(30)


In [ ]:
cellular_automaton_function(90)

In [ ]:
cellular_automaton_function(110)


# Game of Life

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from IPython.display import HTML
import random
from datetime import datetime
import os

def game_of_life(groups=40, generations=100, num_cells=80):
    # We define the initial condition (several patterns)
    cells = np.zeros((num_cells, num_cells))
    patterns = {
        "glider": np.array([[0, 1, 0], [0, 0, 1], [1, 1, 1]]),
        "block": np.array([[1, 1], [1, 1]]),
        "blinker": np.array([[1, 1, 1]]),
        "boat": np.array([[1, 1, 0], [1, 0, 1], [0, 1, 0]]),
        "frog": np.array([[1, 1, 1, 0, 0, 1, 1, 1]]),
        "plane": np.array([[0, 1, 0], [1, 1, 1], [1, 0, 1], [0, 1, 0]]),
        "lightweight_spaceship": np.array([[0, 0, 1, 1, 0], [1, 1, 0, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 0, 0]])
    }
    for _ in range(groups):
        # We choose a random pattern and position
        pattern = random.choice(list(patterns.values()))
        x, y = np.random.randint(0, num_cells - max(pattern.shape), size=2)
        cells[x:x+pattern.shape[0], y:y+pattern.shape[1]] = pattern

    # We create a matrix to store all generations
    generations_arr = np.zeros((generations, num_cells, num_cells))

    def generation_step(cells):
        new_cells = np.zeros((num_cells, num_cells))
        for j in range(1, num_cells-1):
            for k in range(1, num_cells-1):
                state = cells[j-1:j+2, k-1:k+2]
                living_neighbours = np.sum(state) - cells[j, k]
                if cells[j, k] == 1 and (living_neighbours == 2 or living_neighbours == 3):
                    new_cells[j, k] = 1
                elif cells[j, k] == 0 and living_neighbours == 3:
                    new_cells[j, k] = 1
        return new_cells

    # We run the cellular automaton
    for i in range(generations):
        generations_arr[i, :, :] = cells
        cells = generation_step(cells)

    # We prepare the figure for animation
    fig = plt.figure(figsize=(6, 6))

    im = plt.imshow(generations_arr[0, :, :], cmap='binary')

    def update(i):
        im.set_array(generations_arr[i, :, :])

    ani = FuncAnimation(fig, update, frames=generations, interval=500)

    # Create directory "imgs" if it does not exist
    if not os.path.exists("imgs"):
        os.makedirs("imgs")

    # Get current time
    now = datetime.now()
    hhmmss = now.strftime("%H%M%S")

    # Filename
    filename = f"imgs/GameOfLife{hhmmss}"

    # Set up the video writer
    writer = FFMpegWriter(fps=30)

    # Save the animation as .mp4 without axes frame
    with writer.saving(fig, f"{filename}.mp4", dpi=300):
        for i in range(generations):
            update(i)
            writer.grab_frame()

    plt.close(fig)

    return HTML(ani.to_jshtml())


In [ ]:
# Start simulation of the game of life
game_of_life(groups=140, generations=200, num_cells=100)


# Cellular Automata Rules AND Game of Life

In [ ]:
import os
from matplotlib.animation import FuncAnimation, FFMpegWriter
import random
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def generate_rule(n):
    n_bin = format(n, '08b')
    combinations = [(1, 1, 1), (1, 1, 0), (1, 0, 1), (1, 0, 0), (0, 1, 1), (0, 1, 0), (0, 0, 1), (0, 0, 0)]
    rule = {combinations[i]: int(n_bin[i]) for i in range(8)}
    return rule

def update_cells(data, num_cells, rule, board, img):
    new_board = board.copy()

    for i in range(num_cells - 1, num_cells // 2, -1):
        for j in range(1, num_cells - 1):
            new_board[i - 1, j] = rule[(board[i, j - 1], board[i, j], board[i, j + 1])]

    for i in range(num_cells // 2 - 1, -1, -1):
        for j in range(1, num_cells - 1):
            neighbors = board[i - 1:i + 2, j - 1:j + 2]
            total = np.sum(neighbors) - board[i, j]
            if board[i, j] == 1:
                if total < 2 or total > 3:
                    new_board[i, j] = 0
            elif total == 3:
                new_board[i, j] = 1

    board[...] = new_board
    img.set_array(board)
    return img,

def create_animation(rule_num, num_cells):
    board = np.zeros((num_cells, num_cells))
    board[-1, num_cells // 2] = 1
    rule = generate_rule(rule_num)

    fig, ax = plt.subplots(figsize=(num_cells / 20, num_cells / 20))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off')

    img = ax.imshow(board, cmap='binary', interpolation='nearest')
    ani = animation.FuncAnimation(fig, update_cells, fargs=(num_cells, rule, board, img), frames=5 * num_cells, interval=100, blit=True)

    # Create directory "imgs" if it does not exist
    if not os.path.exists("imgs"):
        os.makedirs("imgs")

    # Get current time
    now = datetime.now()
    hhmmss = now.strftime("%H%M%S")

    # Filename
    filename = f"imgs/{rule_num}-Animation{hhmmss}"

    # Set up the video writer
    writer = animation.FFMpegWriter(fps=30)

    # Save the animation as .mp4 without axes frame
    with writer.saving(fig, f"{filename}.mp4", dpi=300):
        for i in range(5 * num_cells):
            update_cells(i, num_cells, rule, board, img)
            writer.grab_frame()

    plt.close(fig)

    # Get the HTML of the video
    video_html = ani.to_jshtml()

    # Display the video in the notebook from the start
    return display(HTML(video_html))


In [ ]:
animation = create_animation(rule_num=30, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=86, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=110, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=124, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=182, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=206, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=214, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=220, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=222, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=238, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=246, num_cells=150)
animation

In [ ]:
animation = create_animation(rule_num=254, num_cells=150)
animation